In [1]:
import torch
import torch.nn as nn
from torch.distributions import Categorical
import numpy as np
from RL import ReinforceAgent
from single_agent_env import SinglePlayerFootball, ACTION_SPACE_SIZE, STATE_SPACE_SIZE
import matplotlib.pyplot as plt
torch.manual_seed(3407)
np.random.seed(3407)

In [2]:
class PG(nn.Module):

    def __init__(self, observation_size, action_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(observation_size, 512),
            nn.LeakyReLU(),
            nn.Linear(512, 256),
            nn.LeakyReLU(),
            nn.Linear(256, 128),
            nn.LeakyReLU(),
            nn.Linear(128, action_size),
            nn.Softmax(dim=0)
        )

    def forward(self, x):
        x = self.model(x)
        m = Categorical(x)
        action = m.sample()
        return action.item(), m.log_prob(action)

In [3]:
env = SinglePlayerFootball(title="Reinforce train")
agent = ReinforceAgent(STATE_SPACE_SIZE, ACTION_SPACE_SIZE, device="cuda:1")
agent.create_model(PG, lr=0.00025, y=0.99)
scores = []

while env.running:
    reward = []
    s = env.reset()
    while not env.loop_once():
        a = agent.policy(s)
        s, r, d = env.step(a)
        agent.learn(r, d)
        reward.append(r)
    scores.append(sum(reward))

env.rendering = True

for _ in range(10):
    s = env.reset()
    while not env.loop_once():
        s, _, _ = env.step(agent.policy(s))

del env

Episode: 1 | Train: 1 | loss: 0.578666
Episode: 2 | Train: 2 | loss: -0.277298
Episode: 3 | Train: 3 | loss: 1.263584
Episode: 4 | Train: 4 | loss: -1.798109
Episode: 5 | Train: 5 | loss: -0.047541
Episode: 6 | Train: 6 | loss: -0.102096
Episode: 7 | Train: 7 | loss: 0.110276
Episode: 8 | Train: 8 | loss: 0.597874
Episode: 9 | Train: 9 | loss: -0.276896
Episode: 10 | Train: 10 | loss: 0.850908
Episode: 11 | Train: 11 | loss: -0.459389
Episode: 12 | Train: 12 | loss: 0.553787
Episode: 13 | Train: 13 | loss: -1.142963
Episode: 14 | Train: 14 | loss: -0.203945
Episode: 15 | Train: 15 | loss: 2.785901
Episode: 16 | Train: 16 | loss: 0.718952
Episode: 17 | Train: 17 | loss: 0.321395
Episode: 18 | Train: 18 | loss: -0.264482
Episode: 19 | Train: 19 | loss: -1.947910
Episode: 20 | Train: 20 | loss: 0.641106
Episode: 21 | Train: 21 | loss: -1.046455
Episode: 22 | Train: 22 | loss: -0.598331
Episode: 23 | Train: 23 | loss: 0.234472
Episode: 24 | Train: 24 | loss: 0.398731
Episode: 25 | Train: 2

In [ ]:
plt.plot(scores)
plt.show()